In [ ]:
"""
This code downloads a videos from Youtube and its related metadata, and
save to Videos folder in the Google Drive
"""

In [11]:
pip install git+https://github.com/mytja/pytube

  Cloning https://github.com/mytja/pytube to /tmp/pip-req-build-p1owlmk1
  Running command git clone -q https://github.com/mytja/pytube /tmp/pip-req-build-p1owlmk1


In [50]:
import pytube
import os
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
root = "/content/drive/My Drive/Let-Hunt-Youtube/"
config_file_path = os.path.join(root, 'config.json')

config = json.load(open(config_file_path))

video_save_folder = config['folder']['video']
thumbnail_save_folder = config['folder']['thumbnail']
database_folder = config['folder']['database']

video_folder_path = os.path.join(root, video_save_folder)
thumbnail_folder_path = os.path.join(root, thumbnail_save_folder)

url_db_file_path = os.path.join(root, database_folder, config['database']['url'])

In [47]:
url_df = pd.read_csv(url_db_file_path)
url_df.sample(5)

,video_url,extracted_date,search_query,video_file_name
32,https://www.youtube.com/watch?v=rfJ5uG8Y3hU,2021-12-08 15:03:53,pcp săn chim,NaN
29,https://www.youtube.com/watch?v=SFHQxDmEsd0,2021-12-08 15:03:53,pcp săn chim,NaN
14,https://www.youtube.com/watch?v=v6xBOVP6PAU,2021-11-23 1:05:42,pcp săn chim,video00015
2,https://www.youtube.com/watch?v=JJoguqMiNF8,2021-11-23 1:05:42,pcp săn chim,video00003
19,https://www.youtube.com/watch?v=Ga8DYVjc2YY,2021-11-23 1:05:42,pcp săn chim,video00020


In [58]:
# get last video name
last_video_name = url_df[url_df.video_file_name.str.len() == 10].video_file_name.to_list()[-1]
# get last video index
last_video_index = int(last_video_name[5:])

for url in tqdm(url_df[url_df.video_file_name.str.len() != 10].video_url.to_list()):
  last_video_index += 1
  youtube = pytube.YouTube(url)
  video_name = f"video{str(last_video_index).zfill(5)}"

  # download video
  try:
    video = youtube.streams.get_by_resolution('720p')
    video.download(video_folder_path, f"{video_name}.mp4")
  except:
    try:
      video = youtube.streams.get_by_resolution('480p')
      video.download(video_folder_path, f"{video_name}.mp4")
    except:
      video = youtube.streams.get_by_resolution('360p')
      video.download(video_folder_path, f"{video_name}.mp4")

  # assign video name in url db
  url_df.iloc[url_df[url_df.video_url == url].index, url_df.columns.get_loc("video_file_name")] = video_name

  # download thumbnail
  thumbnail_img = requests.get(youtube.thumbnail_url).content
  thumbnail_file_path = os.path.join(thumbnail_folder_path, f"thumbnail{str(last_video_index).zfill(5)}.jpeg")
  with open(thumbnail_file_path, 'wb') as handler:
      handler.write(thumbnail_img)


100%|██████████| 5/5 [01:51<00:00, 22.25s/it]


In [59]:
url_df.tail(5)

,video_url,extracted_date,search_query,video_file_name
31,https://www.youtube.com/watch?v=1mQEYg_ELtw,2021-12-08 15:03:53,pcp săn chim,video00032
32,https://www.youtube.com/watch?v=rfJ5uG8Y3hU,2021-12-08 15:03:53,pcp săn chim,video00033
33,https://www.youtube.com/watch?v=umLittCtnZo,2021-12-08 15:03:53,pcp săn chim,video00034
34,https://www.youtube.com/watch?v=3Csv6q1qqvA,2021-12-08 15:03:53,pcp săn chim,video00035
35,https://www.youtube.com/watch?v=uYA5tU0AADU,2021-12-08 15:03:53,pcp săn chim,video00036


In [61]:
# assert before saving
assert 1==1
# save database
url_df.to_csv(url_db_file_path, index=False)